In [1]:
import gzip
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
variant_disease = pd.read_csv("../SAR/all_variant_disease_associations.tsv.gz", compression='gzip', sep = '\t')

In [3]:
# Find GWAS genes in the database
RA = list()
for i in range(len(variant_disease['diseaseName'])):
    if 'rheumatoid' in variant_disease['diseaseName'][i]:
        RA.append(i)
    if 'Rheumatoid' in variant_disease['diseaseName'][i]:
        RA.append(i)

    

In [4]:
np.unique(variant_disease['diseaseName'][RA])

array(['Early Rheumatoid Arthritis', 'Juvenile rheumatoid arthritis',
       'Polyarticular Juvenile Idiopathic Arthritis, Rheumatoid Factor Negative',
       'Progression of rheumatoid arthritis',
       'Progressive pseudorheumatoid dysplasia', 'Rheumatoid Arthritis',
       'Rheumatoid Arthritis, Systemic Juvenile',
       'Rheumatoid Factor Measurement', 'Rheumatoid Nodule',
       'Seropositive rheumatoid arthritis'], dtype=object)

In [5]:
# Let's say that we don't want Pediatric Crohn's disease, we can get rid of it for example the following way
RA = list()
for i in range(len(variant_disease['diseaseName'])):
    if 'Rheumatoid Arthritis' in variant_disease['diseaseName'][i]\
    and 'Juvenile' not in variant_disease['diseaseName'][i]\
    and 'Early' not in variant_disease['diseaseName'][i]:
        RA.append(i)

In [6]:
#Now we are satisfied with the diseases
np.unique(variant_disease['diseaseName'][RA])

array(['Rheumatoid Arthritis'], dtype=object)

In [7]:
#These are information about the diseases from the database
variant_disease.loc[RA]

,snpId,chromosome,position,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,source
608,rs1003878,6,32332045,0.882,0.20,C0003873,Rheumatoid Arthritis,disease,C17;C05;C20,Disease or Syndrome,0.70,1.0,2007.0,2011.0,3,GWASDB;INFERRED
611,rs1003879,6,32331815,1.000,0.12,C0003873,Rheumatoid Arthritis,disease,C17;C05;C20,Disease or Syndrome,0.70,1.0,2011.0,2011.0,1,GWASDB;INFERRED
758,rs1004819,1,67204530,0.776,0.36,C0003873,Rheumatoid Arthritis,disease,C17;C05;C20,Disease or Syndrome,0.01,1.0,2013.0,2013.0,1,BEFREE
809,rs1005133,22,19750832,1.000,0.12,C0003873,Rheumatoid Arthritis,disease,C17;C05;C20,Disease or Syndrome,0.70,1.0,2019.0,2019.0,1,GWASCAT;INFERRED
916,rs1005753,1,17118274,1.000,0.12,C0003873,Rheumatoid Arthritis,disease,C17;C05;C20,Disease or Syndrome,0.01,1.0,2012.0,2012.0,1,BEFREE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367688,rs9880772,3,27736288,0.827,0.24,C0003873,Rheumatoid Arthritis,disease,C17;C05;C20,Disease or Syndrome,0.70,1.0,2015.0,2015.0,1,GWASCAT;INFERRED
368137,rs9909240,17,9236774,1.000,0.12,C0003873,Rheumatoid Arthritis,disease,C17;C05;C20,Disease or Syndrome,0.70,1.0,2009.0,2009.0,1,GWASDB;INFERRED
368286,rs991760,6,32855790,1.000,0.12,C0003873,Rheumatoid Arthritis,disease,C17;C05;C20,Disease or Syndrome,0.70,1.0,2009.0,2009.0,1,GWASDB;INFERRED
369265,rs9979383,21,35343463,0.925,0.20,C0003873,Rheumatoid Arthritis,disease,C17;C05;C20,Disease or Syndrome,0.80,1.0,2012.0,2016.0,3,GWASCAT;GWASDB;INFERRED


In [8]:
#Map the database to the geneSymbol names
snpId = variant_disease.loc[RA]['snpId']
variant_gene = pd.read_csv("../SAR/variant_to_gene_mappings.tsv.gz", compression='gzip', sep = '\t')
chosen = list()
for i in range(len(variant_gene['snpId'])):
    if variant_gene['snpId'][i] in np.array(snpId):
        chosen.append(i)
variant_gene.loc[chosen]

,snpId,geneId,geneSymbol,sourceId
29,rs1801280,10,NAT2,VEP
30,rs1799930,10,NAT2,VEP
32,rs1799931,10,NAT2,DBSNP
33,rs1799931,10,NAT2,VEP
34,rs1799930,10,NAT2,DBSNP
...,...,...,...,...
361465,rs629326,112267968,LOC112267968,DBSNP
361466,rs212389,112267968,LOC112267968,DBSNP
361469,rs2451258,112267968,LOC112267968,DBSNP
361916,rs2395379,113523636,SMIM40,DBSNP


In [9]:
# Here is the list of GWAS genes that we are interested in
GWAS_RA = np.unique(variant_gene.loc[chosen]['geneSymbol'])
len(GWAS_RA)

777

In [10]:
pathways_all = pd.read_csv('PathInfo_scRNAseq.csv') 
pathways_cluster1 = pd.read_excel('TreeStructure_nodes2_scRNAseq.xlsx') 

In [11]:
all_genes = pd.read_csv('../../../../../data/sharedData/CIA_project/DGE_data/sorted_DGEs/allTissues_sorted_expression_matrix.csv', sep = ',', index_col = 0)
translation = pd.read_csv('../../../../../data/sharedData/CIA_project/results/orthologous_translation/orthologous_translation_NicheNet_analysis.txt', sep = '\t')

In [12]:
dataset_DEGs = !ls '../../../../../data/sharedData/CIA_project/results/DEG_analysis/cluster_ids_fromOleg_06_29_CellType/change_mode/fdr_sorted/'
dataset_DEGs = dataset_DEGs[4:]

In [13]:
dataset_DEGs

['Number_of_genes_significant_DEGs_Sick_vs_Healthy.csv',
 'significant_DEGs_Sick_vs_Healthy_B-cells_Muscle.csv',
 'significant_DEGs_Sick_vs_Healthy_B-cells_Spleen.csv',
 'significant_DEGs_Sick_vs_Healthy_Dendritic-cells_Muscle.csv',
 'significant_DEGs_Sick_vs_Healthy_Dendritic-cells_Spleen.csv',
 'significant_DEGs_Sick_vs_Healthy_Endothelial-cells_Joint.csv',
 'significant_DEGs_Sick_vs_Healthy_Endothelial-cells_Lung.csv',
 'significant_DEGs_Sick_vs_Healthy_Endothelial-cells_Muscle.csv',
 'significant_DEGs_Sick_vs_Healthy_Endothelial-cells_Skin.csv',
 'significant_DEGs_Sick_vs_Healthy_Erythrocytes_Muscle.csv',
 'significant_DEGs_Sick_vs_Healthy_Erythrocytes_Spleen.csv',
 'significant_DEGs_Sick_vs_Healthy_Fibroblasts_Joint.csv',
 'significant_DEGs_Sick_vs_Healthy_Fibroblasts_Lung.csv',
 'significant_DEGs_Sick_vs_Healthy_Fibroblasts_Muscle.csv',
 'significant_DEGs_Sick_vs_Healthy_Fibroblasts_Skin.csv',
 'significant_DEGs_Sick_vs_Healthy_Granulocytes_Joint.csv',
 'significant_DEGs_Sick_vs_

In [30]:


DEGs_from_pathway = list()
tissue = list()
cell_type = list()
DEGs_list = list()
path = '../../../../../data/sharedData/CIA_project/results/DEG_analysis/cluster_ids_fromOleg_06_29_CellType/change_mode/fdr_sorted/'
for subset in dataset_DEGs: #Loop through all DEG files
    #if subset.split('_')[6].split('.')[0] == 'Muscle' or subset.split('_')[6].split('.')[0] == 'Joint':
    #if subset.split('_')[6].split('.')[0] == 'Muscle':
    if subset.split('_')[6].split('.')[0] == 'Joint':

        subset = subset.split(' ')[-1]
        DEGs_subset = pd.read_csv(path + subset, index_col = 0) #Read file 
        #Translate to human names
        DEGs_subset = translation['Gene.name'][np.where(translation['Mouse.gene.name'].isin(DEGs_subset.index))[0]] 


        #Pick the shared genes between pathway and DEGs file
        DEGs_list.append(DEGs_subset) 

        #to remember the tissue and cell type name
        tissue.append(subset.split('_')[6].split('.')[0])
        cell_type.append(subset.split('_')[5])

In [35]:
"""Cluster 1"""
pathways = pathways_all.loc[np.where(pathways_all['IngenuityCanonicalPathways'].isin\
                          (pathways_cluster1['IngenuityCanonicalPathways']))[0]]
"""Cluster 2"""
#pathways = pathways_all.loc[np.where(~pathways_all['IngenuityCanonicalPathways'].isin\
#                          (pathways_cluster1['IngenuityCanonicalPathways']))[0]]


pathways.index = np.array(range(len(pathways.index)))

odds_list = list()
pval_list = list()
for ct in np.unique(cell_type):
    idx = np.where(np.array(cell_type) == ct)[0]
    if len(idx) == 1:
        DEGs_cell_type = np.array(DEGs_list[int(idx)])
    else:
        DEGs_cell_type = np.union1d(DEGs_list[int(idx[0])], DEGs_list[int(idx[1])])
        
    DEGs = list()
    for k in range(len(pathways.index)): 
        p = pathways.loc[k]
        DEGs.append(str(p['AllMolecules']).split(','))
    DEGs = np.concatenate(DEGs)
    DEGs = np.unique(DEGs)

    background_genes = np.unique(DEGs_cell_type)
    DEGs = np.intersect1d(DEGs, background_genes)
    GWAS_RA = np.unique(GWAS_RA)
    nonDEGs = list(set(background_genes) - set(DEGs))                
    table11 = len(np.intersect1d(GWAS_RA, DEGs))
    table12 = len(DEGs) - table11
    table21 = len(np.intersect1d(GWAS_RA, nonDEGs))
    table22 = len(background_genes) - (table21 + table12 + table11)
    table = pd.DataFrame([[table11, table12], [table21, table22]])
    table.index = ('DEGs', 'nonDEGs')
    table.columns = ('GWAS', 'nonGWAS')
    odds, pval = stats.fisher_exact(table, alternative = 'greater')
    odds_list.append(odds)
    pval_list.append(pval)
    

In [36]:
GWAS_summary = pd.DataFrame((odds_list, pval_list), columns = np.unique(cell_type), index = ('odds', 'pval')).transpose()

In [37]:
GWAS_summary = GWAS_summary.sort_values('pval')
GWAS_summary

,odds,pval
Endothelial-cells,3.455877,9.556580e-08
Unknown6,3.070061,3.358217e-07
Fibroblasts,3.726646,2.990161e-06
Granulocytes,2.798690,1.950336e-05
Macrophages,2.387516,3.784856e-04
T-cells,2.084058,3.186629e-03


In [38]:
#GWAS_summary.to_csv('Joint_GWAS_summary_scRNA_cluster1_oneSided.csv', index = True, header = True)

In [39]:
odds_list2 = list()
pval_list2 = list()

for ct in np.unique(cell_type):
    odds_list = list()
    pval_list = list()
    idx = np.where(np.array(cell_type) == ct)[0]
    if len(idx) == 1:
        DEGs_cell_type = np.array(DEGs_list[int(idx)])
    else:
        DEGs_cell_type = np.union1d(DEGs_list[int(idx[0])], DEGs_list[int(idx[1])])

    for j in np.unique(pathways_cluster1['subclusters']):
        pathways_subset = pathways_cluster1.loc[np.where(pathways_cluster1['subclusters'] == j)[0]]
        pathways_subset.index = np.array(range(len(pathways_subset.index)))
        DEGs = list()
        for k in range(len(pathways_subset.index)): 
            p = pathways_subset.loc[k]
            DEGs.append(str(p['AllMolecules']).split(','))
        DEGs = np.concatenate(DEGs)
        DEGs = np.unique(DEGs)
        
        background_genes = np.unique(DEGs_cell_type)
        DEGs = np.intersect1d(DEGs, background_genes)
    
        GWAS_RA = np.unique(GWAS_RA)
        nonDEGs = list(set(background_genes) - set(DEGs))                
        table11 = len(np.intersect1d(GWAS_RA, DEGs))
        table12 = len(DEGs) - table11
        table21 = len(np.intersect1d(GWAS_RA, nonDEGs))
        table22 = len(background_genes) - (table21 + table12 + table11)
        table = pd.DataFrame([[table11, table12], [table21, table22]])
        table.index = ('DEGs', 'nonDEGs')
        table.columns = ('GWAS', 'nonGWAS')
        odds, pval = stats.fisher_exact(table, alternative = 'greater')
        odds_list.append(odds)
        pval_list.append(pval)
    odds_list2.append(odds_list)
    pval_list2.append(pval_list)

# if the odds are > 1 and pval <0.05 The list of DEGs is significantly enriched by the GWAS genes
# if the odds are < 1 and pval <0.05 The list of DEGs is not enriched by the GWAS genes

In [40]:
Odds = pd.DataFrame(odds_list2, index = np.unique(cell_type), columns = np.array(range(10)) + 1)

In [41]:
Odds

,1,2,3,4,5,6,7,8,9,10
Endothelial-cells,3.122826,3.478670,2.856621,3.619524,3.353654,5.921626,3.922292,2.629849,3.526307,3.141188
Fibroblasts,3.274154,4.644444,5.377778,3.953692,4.830508,6.300000,6.156305,1.896226,6.509255,6.504000
Granulocytes,3.102975,3.682917,2.862599,2.255319,4.086397,4.812865,3.833618,1.260522,2.860496,5.886667
Macrophages,3.545235,2.518050,3.709013,2.043735,2.734013,4.858580,3.193212,2.084507,4.204882,4.208791
T-cells,2.262338,2.068748,1.087926,1.979545,1.254305,3.435364,2.408565,1.505017,1.991741,3.053512
Unknown6,3.665166,3.808571,4.905469,3.983192,5.257104,4.973844,3.704710,2.683774,3.517480,4.363454


In [42]:
#Odds.to_csv('Joint_Odds_summary_scRNA_oneSided.csv', index = True, header = True)

In [43]:
Pval = pd.DataFrame(pval_list2, index = np.unique(cell_type), columns = np.array(range(10)) + 1)

In [44]:
Pval

,1,2,3,4,5,6,7,8,9,10
Endothelial-cells,0.001104,3.064935e-06,6.284407e-03,0.000208,6.206997e-04,3.655343e-11,2.281222e-05,0.057082,0.000410,0.052866
Fibroblasts,0.004825,6.969792e-07,5.767664e-05,0.001606,7.036155e-05,1.471196e-08,2.521950e-07,0.236429,0.000003,0.001051
Granulocytes,0.001657,3.704884e-06,1.278130e-02,0.030734,1.118395e-04,4.060871e-08,4.482997e-05,0.440616,0.006091,0.001409
Macrophages,0.000409,1.393803e-03,7.399271e-04,0.061872,8.334082e-03,7.876349e-08,9.620009e-04,0.148365,0.000096,0.011896
T-cells,0.030413,1.572549e-02,5.134017e-01,0.069584,3.870609e-01,4.660586e-05,1.298656e-02,0.339413,0.082911,0.091669
Unknown6,0.000024,1.067897e-07,1.126156e-07,0.000006,2.157786e-08,3.638438e-10,3.764212e-06,0.016853,0.000093,0.001269


In [45]:
#Pval.to_csv('Joint_Pval_summary_scRNA_oneSided.csv', index = True, header = True)